In [8]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import pandas as pd
import tensorflow as tf
import nltk
import regex as re

from google.colab import drive
drive.mount('/content/drive')
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
! cp drive/My\ Drive/JobDescriptionPrediction/Resources/Model/* .
!tar -xvf  'checkpoint_run1.tar' -C .
!tar -xvf  'checkpoint_run2.tar' -C .
!tar -xvf  'checkpoint_run3.tar' -C .
# gpt2.copy_checkpoint_from_gdrive(run_name="run1")
# gpt2.copy_checkpoint_from_gdrive(run_name="run2")
# gpt2.copy_checkpoint_from_gdrive(run_name="run3")

cp: -r not specified; omitting directory 'drive/My Drive/JobDescriptionPrediction/Resources/Model/T5-Aboutcompany'
cp: -r not specified; omitting directory 'drive/My Drive/JobDescriptionPrediction/Resources/Model/T5-Role'
cp: -r not specified; omitting directory 'drive/My Drive/JobDescriptionPrediction/Resources/Model/T5-Skill'
checkpoint/run1/
checkpoint/run1/checkpoint
checkpoint/run1/counter
checkpoint/run1/encoder.json
checkpoint/run1/events.out.tfevents.1617255936.c51996cb59be
checkpoint/run1/hparams.json
checkpoint/run1/model-3337.data-00000-of-00001
checkpoint/run1/model-3337.index
checkpoint/run1/model-3337.meta
checkpoint/run1/vocab.bpe
checkpoint/run2/
checkpoint/run2/checkpoint
checkpoint/run2/counter
checkpoint/run2/encoder.json
checkpoint/run2/events.out.tfevents.1617285150.0c4597a381ce
checkpoint/run2/hparams.json
checkpoint/run2/model-1000.data-00000-of-00001
checkpoint/run2/model-1000.index
checkpoint/run2/model-1000.meta
checkpoint/run2/vocab.bpe
checkpoint/run3/
check

In [10]:
def runModelSingleInput(run_name, input):
  print( 'inside run_Model')
  tf.reset_default_graph()
  sess = gpt2.start_tf_sess()
  gpt2.load_gpt2(sess, run_name=run_name)
  print( run_name, ' :: Model loaded ')
  output = predict(sess,run_name,input)
  return output
def runModelMultipleInputs(run_name, input):
  print( 'inside run_Model')
  tf.reset_default_graph()
  sess = gpt2.start_tf_sess()
  gpt2.load_gpt2(sess, run_name=run_name)
  print( run_name, ' :: Model loaded ')
  output= []
  for i in input:
    print('input i ::',i)
    output.append(predict(run_name,i))
  return output

In [11]:
def predict(sess,run_name,input):
  output = gpt2.generate(sess,
                         run_name = run_name,
                  prefix=input,
                  length=100,
                  temperature=0.7,
                  nsamples=1,
                  return_as_list=True
                  )
  output = output[0].replace(input,'')
  output = output.replace('\', \'','')
  output = output.replace('[\'','')
  output = output.replace('[','')
  output = output.replace(']','')
  output = output.replace('.,','.')
  output = output.replace('"','.')
  output = re.sub('\s+',' ',output)
  output = re.sub('^,','',output)
  print( run_name, ' :: Output generated ')
  return output

In [12]:
def getModelOutputSingleInput(Input_Job_title,Input_Skills,Input_Company_name,Input_Location):
  text_skills = runModelSingleInput('run1',Input_Job_title+ ' '+Input_Skills)
  
  text_role = runModelSingleInput('run2',Input_Job_title+ ' '+Input_Skills)
  
  text_aboutCompany = runModelSingleInput('run3',Input_Company_name +' '+Input_Location)
  
  output = 'SKILLS'+'\n'+text_skills + '\n\n'+ 'ROLES'+'\n'+text_role+ '\n\n'+ 'ABOUT COMPANY'+'\n'+text_aboutCompany
  return output

In [13]:
def getModelOutputMultipleInputs(Input_Job_title,Input_Skills,Input_Company_name,Input_Location):

  text_skills = runModelMultipleInputs('run1',Input_Job_title+ ' '+Input_Skills)
  
  text_role = runModelMultipleInputs('run2',Input_Job_title+ ' '+Input_Skills)
  
  text_aboutCompany = runModelMultipleInputs('run3',Input_Company_name +' '+Input_Location)
  
  output = 'SKILLS'+'\n'+text_skills + '\n\n'+ 'ROLES'+'\n'+text_role+ '\n\n'+ 'ABOUT COMPANY'+'\n'+text_aboutCompany
  return output

In [14]:
# API inputs
Input_Job_title = 'data scientist'
Input_Skills = 'sap sql'
Input_Company_name = 'express scripts'
Input_Location = 'MO'

#API to cALL this method, getMOdelOutputs

output  = getModelOutputSingleInput(Input_Job_title,Input_Skills,Input_Company_name,Input_Location)

# API OUTPUT 
print(output)

inside run_Model
Loading checkpoint checkpoint/run1/model-3337
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-3337
run1  :: Model loaded 
run1  :: Output generated 
inside run_Model
Loading checkpoint checkpoint/run2/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run2/model-1000
run2  :: Model loaded 
run2  :: Output generated 
inside run_Model
Loading checkpoint checkpoint/run3/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run3/model-1000
run3  :: Model loaded 
run3  :: Output generated 
SKILLS
 design, validate, implement, and perform predictive analysis and data mining on a wide variety of datasets work closely with product managers to discover and build dashboards and scoredigest meaningful insights identify meaningful insights that add up to reward growth develop metrics and kpis to track student performance vs. forecast forecast abuse create dashboards and reports to continuously analyze critical metrics and product kpis train a